<h1 style='text-align:center;color:slategray'>XGBoost Hyperparameter Tuning</h1>

In [18]:
# imports
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timedelta

In [1]:
# Replace <PROJECT_ID>, <BUCKET_ID>, and <FOLDER> with proper Project, Bucket ID, and Folder.
# project = '<PROJECT_ID>'
# bucket = '<BUCKET_ID>'
# folder = '<FOLDER>'
# region = 'us-central1'

project = 'mwpmltr'
bucket = 'bjacob_bucket'
folder = 'census_income'
region = 'us-central1'

In [2]:
bucket_path=f'{bucket}/{folder}'
%env PROJECT_ID=$project
%env BUCKET_ID=$bucket
%env BUCKET_PATH=$bucket_path
!gsutil mb -c standard -l {region} gs://{bucket}

env: PROJECT_ID=mwpmltr
env: BUCKET_ID=bjacob_bucket
env: BUCKET_PATH=bjacob_bucket/census_income
Creating gs://bjacob_bucket/...
ServiceException: 409 Bucket bjacob_bucket already exists.


In [10]:
%env REGION us-central1
%env TRAINER_PACKAGE_PATH ./trainer
%env MAIN_TRAINER_MODULE trainer.task_xgboost
%env RUNTIME_VERSION=2.2
%env PYTHON_VERSION=3.7
%env HPTUNING_CONFIG trainer/hptuning_config_xgboost.yaml

env: REGION=us-central1
env: TRAINER_PACKAGE_PATH=./trainer
env: MAIN_TRAINER_MODULE=trainer.task_xgboost
env: RUNTIME_VERSION=2.2
env: PYTHON_VERSION=3.7
env: HPTUNING_CONFIG=trainer/hptuning_config_xgboost.yaml


In [13]:
now=(datetime.now() + timedelta(hours=-5)).strftime("%Y%m%d_%H%M%S") # Eastern Time
%env JOB_NAME=census_income_xgboost_job_{now}

!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir gs://${BUCKET_PATH}/jobdir \
  --package-path $TRAINER_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --scale-tier basic \
  --config $HPTUNING_CONFIG \
  -- \
  --metric auc \
  --run_type cloud

env: JOB_NAME=census_income_xgboost_job_20201207_182332
Job [census_income_xgboost_job_20201207_182332] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe census_income_xgboost_job_20201207_182332

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs census_income_xgboost_job_20201207_182332
jobId: census_income_xgboost_job_20201207_182332
state: QUEUED


In [14]:
print('Waiting for AI-Platform job to complete ...')
job_outpt = !gcloud ai-platform jobs describe --format json $JOB_NAME
outpt_dict = json.loads(job_outpt.get_spstr())
while outpt_dict['state'] not in ['SUCCEEDED', 'FAILED', 'CANCELLED', 'CANCELLING']:
    time.sleep(30)
    job_outpt = !gcloud ai-platform jobs describe --format json $JOB_NAME
    outpt_dict = json.loads(job_outpt.get_spstr())
if outpt_dict['state'] != 'SUCCEEDED':
    !gcloud ai-platform jobs describe $JOB_NAME
    raise(Exception('There was an issue with the AI-Platform Job. Please see the logs for information.'))
print('Finished.')

Waiting for AI-Platform job to complete ...
Finished.


In [19]:
trials = outpt_dict['trainingOutput']['trials']
scores = [float(t['finalMetric']['objectiveValue']) for t in trials]
trial_ids = [t['trialId'] for t in trials]
hypers = pd.DataFrame([t['hyperparameters'] for t in trials], dtype=np.float64)
max_idx = np.argmax(scores)
job_df = pd.DataFrame({'Trial_ID': trial_ids, 'ROC_AUC': scores})
job_df = job_df.merge(hypers, left_index=True, right_index=True).set_index('Trial_ID')
pd.options.display.float_format = '{:,.4f}'.format
for col in job_df.columns:
    if col in ['batch_size', 'hidden_depth', 'hidden_nodes', 'lr_decay_param']:
        job_df[col] = job_df[col].astype('int')
    job_df[col] = job_df[col].astype('object')  # force correct display of ints and floats after transpose
job_df.head(5).T

Trial_ID,2,1
ROC_AUC,0.9254,0.9242
class_weight_ratio,2.3728,2.5000
learning_rate,0.4408,0.3250
max_depth,8.0000,11.0000
num_boost_round,60.0000,60.0000


<h1 style='text-align:center;color:slategray'>Tensorflow Hyperparameter Tuning</h1>

In [34]:
%env REGION us-central1
%env TRAINER_PACKAGE_PATH ./trainer
%env MAIN_TRAINER_MODULE trainer.task_tensorflow
%env RUNTIME_VERSION=2.2
%env PYTHON_VERSION=3.7
%env HPTUNING_CONFIG trainer/hptuning_config_tensorflow.yaml

env: REGION=us-central1
env: TRAINER_PACKAGE_PATH=./trainer
env: MAIN_TRAINER_MODULE=trainer.task_tensorflow
env: RUNTIME_VERSION=2.2
env: PYTHON_VERSION=3.7
env: HPTUNING_CONFIG=trainer/hptuning_config_tensorflow.yaml


In [35]:
now=(datetime.now() + timedelta(hours=-5)).strftime("%Y%m%d_%H%M%S") # Eastern Time
%env JOB_NAME=census_income_tensorflow_job_{now}

!gcloud ai-platform jobs submit training $JOB_NAME \
  --job-dir gs://${BUCKET_PATH}/jobdir \
  --package-path $TRAINER_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version $RUNTIME_VERSION \
  --python-version $PYTHON_VERSION \
  --scale-tier basic \
  --config $HPTUNING_CONFIG \
  -- \
  --metric val_auc \
  --epochs 1 \
  --run_type 'cloud'

env: JOB_NAME=census_income_tensorflow_job_20201207_200629
Job [census_income_tensorflow_job_20201207_200629] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe census_income_tensorflow_job_20201207_200629

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs census_income_tensorflow_job_20201207_200629
jobId: census_income_tensorflow_job_20201207_200629
state: QUEUED


In [36]:
print('Waiting for AI-Platform job to complete ...')
job_outpt = !gcloud ai-platform jobs describe --format json $JOB_NAME
outpt_dict = json.loads(job_outpt.get_spstr())
while outpt_dict['state'] not in ['SUCCEEDED', 'FAILED', 'CANCELLED', 'CANCELLING']:
    time.sleep(30)
    job_outpt = !gcloud ai-platform jobs describe --format json $JOB_NAME
    outpt_dict = json.loads(job_outpt.get_spstr())
if outpt_dict['state'] != 'SUCCEEDED':
    !gcloud ai-platform jobs describe $JOB_NAME
    raise(Exception('There was an issue with the AI-Platform Job. Please see the logs for information.'))
print('Finished.')

Waiting for AI-Platform job to complete ...
Finished.


In [37]:
trials = outpt_dict['trainingOutput']['trials']
scores = [float(t['finalMetric']['objectiveValue']) for t in trials]
trial_ids = [t['trialId'] for t in trials]
hypers = pd.DataFrame([t['hyperparameters'] for t in trials], dtype=np.float64)
max_idx = np.argmax(scores)
job_df = pd.DataFrame({'Trial_ID': trial_ids, 'ROC_AUC': scores})
job_df = job_df.merge(hypers, left_index=True, right_index=True).set_index('Trial_ID')
pd.options.display.float_format = '{:,.4f}'.format
for col in job_df.columns:
    if col in ['batch_size', 'hidden_depth', 'hidden_nodes', 'lr_decay_param']:
        job_df[col] = job_df[col].astype('int')
    job_df[col] = job_df[col].astype('object')  # force correct display of ints and floats after transpose
job_df.head(5).T

Trial_ID,1,2
ROC_AUC,0.9096,0.9052
batch_size,64,32
class_weight_ratio,2.5000,2.0950
hidden_activation,selu,elu
hidden_depth,2,1
hidden_nodes,32,16
initial_lr,0.0051,0.0031
lr_decay_param,55,66
optimizer,RMSprop,Adam
